# Анализ базы данных приложения для чтения книг.

В базе содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

__Цель исследования:__ Сформулировать ценностное предложение для нового продукта.

__Описание данных:__
- Таблица books
Содержит данные о книгах:
        book_id — идентификатор книги;
        author_id — идентификатор автора;
        title — название книги;
        num_pages — количество страниц;
        publication_date — дата публикации книги;
        publisher_id — идентификатор издателя.
- Таблица authors
Содержит данные об авторах:
        author_id — идентификатор автора;
        author — имя автора.
- Таблица publishers
Содержит данные об издательствах:
        publisher_id — идентификатор издательства;
        publisher — название издательства;
- Таблица ratings
Содержит данные о пользовательских оценках книг:
        rating_id — идентификатор оценки;
        book_id — идентификатор книги;
        username — имя пользователя, оставившего оценку;
        rating — оценка книги.
- Таблица reviews
Содержит данные о пользовательских обзорах:
        review_id — идентификатор обзора;
        book_id — идентификатор книги;
        username — имя автора обзора;
        text — текст обзора.



In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# выведем информацию о таблицах
# таблица authors
query = ''' 
SELECT *
FROM authors
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
# таблица books
query = ''' 
SELECT *
FROM books
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# таблица publishers
query = ''' 
SELECT *
FROM publishers
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# таблица ratings
query = ''' 
SELECT *
FROM ratings
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# таблица reviews
query = ''' 
SELECT *
FROM reviews
LIMIT 5;
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### 1.	Посчитайте, сколько книг вышло после 1 января 2000 года

In [7]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT COUNT(book_id)
FROM books
WHERE  publication_date > '2000-01-01';
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


- Всего после 01.01.2000г. вышло 819 книг.

### 2.	Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
query = '''SELECT b.book_id,
                    b.title,
                    COUNT(DISTINCT r.review_id) as review_count,
                    AVG(ra.rating) as avg_rating
        FROM books as b
        LEFT JOIN reviews as r ON b. book_id = r. book_id
        LEFT JOIN ratings as ra ON r. book_id = ra.book_id
        GROUP BY b.book_id
        ORDER BY 3 DESC, 4 DESC
        LIMIT 10;
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,review_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


- Самое большое число обзоров у "Сумерек (часть 1)" (Twilight (Twilight #1)), тем не менее средний рейнг у них не самый высокий, далее в тройку самых часто комментиуремых книг входят книги о Гарри Поттере: Узник Азкабана и Тайная комната - 2 и 3 место соответственно. Также у эти книги лидируют по сумме средней оценки (рейтинг).

### 3.	Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [9]:
query = '''SELECT p.publisher, COUNT(b.book_id)
FROM publishers as p
RIGHT JOIN books as b ON p.publisher_id = b. publisher_id
WHERE b.num_pages > 50
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42


- Наибольшее число книг толщиной более 50 страниц выпустило издательство Penguin Books - 42 книги.

### 4.	Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [10]:
query = '''WITH temp as
(SELECT b.author_id, COUNT(b.book_id) as book_count, AVG(r.rating) as avg_rating
FROM books as b
JOIN ratings as r ON b. book_id = r. book_id
GROUP BY b.author_id, b.book_id
HAVING COUNT(r.rating) >= 50
ORDER BY avg(r.rating) DESC)

SELECT a.author, AVG(temp.avg_rating) as avg_rating
FROM authors as a
JOIN temp ON a.author_id = temp.author_id
GROUP BY 1
LIMIT 1; 

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844


- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré со средней оценкой 4,28.

### 5.	Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [11]:
query = '''SELECT AVG(review_count)
FROM (
SELECT username, COUNT(review_id) as review_count
FROM reviews
WHERE username IN (SELECT username
FROM ratings
GROUP BY 1
HAVING COUNT(rating_id) > 48) 
GROUP BY 1) as temp_1
;
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 штуки.

__Выводы:__
- Всего в базе данные по 819 книгам, вышедшим после 01.01.2000г.
- Самое большое число обзоров у "Сумерек (часть 1)", тем не менее средний рейнг у них не самый высокий, далее в тройку самых часто комментиуремых книг входят книги о Гарри Поттере: Узник Азкабана и Тайная комната - 2 и 3 место соответственно. Также у эти книги лидируют по сумме средней оценки (рейтинг).
- Издательство "Penguin Books" выпустило наибольшее количество книг толще 50 страниц - 42 шт.
- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré со средней оценкой 4,28.
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24 штуки.

    Ожидаемо в топах книг и авторов до сих пор остаются Дж.Роулинг и книги о Гарри Поттере. Похоже читатели в целом чаще предпочитают фэнтези: в топе книг с оценкой выше 4 - Сумерки, Хоббит и приключеняи Перси Джексона.    